In [1]:
%matplotlib inline

In [13]:
# stdlib imports
from datetime import datetime

# third party libraries
import pandas as pd
import numpy as np

# local libraries
from libcomcat.dataframes import associate

# Associating Earthquake Catalogues

A fairly common task at the GHSC is associating catalogues of earthquakes from various sources with events in ComCat. We've created an *associate()* function which should automate a good portion of this process, minus data cleaning. This notebook will hopefully address some of the more common issues involved with data cleaning, including:

 - Dealing with missing data
 - Converting date/time values to pandas datetime/timestamp objects
 - Adjusting time/distance/magnitude tolerances to accomodate uncertainty

In [3]:
datafile = '../tests/data/sample_catalogue.csv'
DMINUTE = 60 # number of seconds in a minute
DHOUR = 3600 # number of seconds in an hour
DDAY = 86400 # number of seconds in a day

## Sample Dataset

This dataset is a severely down-sampled (in both rows and columns) NOAA macroseismic intensity table. It has a couple of notable issues that need to be dealt with:

 - Date and time information is spread out through six different columns
 - Some of that date and time information is missing

In [4]:
dataframe = pd.read_csv(datafile)
dataframe

,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,MAGNITUDE,LATITUDE,LONGITUDE,CITY,MMI
0,1904,8.0,27.0,21.0,56.0,6.0,8.3,65.48,-150.30,RAMPART,6.0
1,1908,5.0,15.0,8.0,31.0,36.0,7.0,59.00,-141.00,CAPE YAKATAGA,6.0
2,1909,9.0,8.0,17.0,49.0,48.0,7.4,52.50,-169.00,ALEUTIAN ISLANDS,5.0
3,1911,9.0,22.0,5.0,1.0,24.0,6.9,60.50,-149.00,PRINCE WILLIAM SOUND,8.0
4,1912,1.0,31.0,20.0,11.0,48.0,7.3,61.00,-147.50,VALDES,6.0
5,1912,6.0,7.0,9.0,55.0,54.0,6.4,59.00,-153.00,KANATAK,5.0
6,1912,6.0,10.0,16.0,6.0,6.0,7.0,59.00,-153.00,NaN,3.0
7,1912,7.0,7.0,7.0,57.0,36.0,7.4,64.00,-147.00,KENNECOTT,5.0
8,1912,7.0,7.0,21.0,57.0,36.0,7.4,64.00,-147.00,FAIRBANKS,6.0
9,1912,11.0,7.0,7.0,40.0,24.0,7.5,57.50,-155.00,SEWARDOTT,NaN


### Deal with missing time data and resulting accuracy issues

First identify the rows where we have missing time data.  We'll create an "accuracy" field based on the most signicant missing time column that we'll use to determine a time tolerance value.

In [5]:
# get the indices of the rows where seconds, minutes, or hours are missing
idx_sec = dataframe['SECOND'].isnull()
idx_minute = dataframe['MINUTE'].isnull()
idx_hour = dataframe['HOUR'].isnull()

# create an accuracy column that reflects the greatest lack of resolution
dataframe['accuracy'] = 0
dataframe.loc[idx_sec, 'accuracy'] = DMINUTE
dataframe.loc[idx_minute, 'accuracy'] = DHOUR
dataframe.loc[idx_hour, 'accuracy'] = DDAY

dataframe

,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,MAGNITUDE,LATITUDE,LONGITUDE,CITY,MMI,accuracy
0,1904,8.0,27.0,21.0,56.0,6.0,8.3,65.48,-150.30,RAMPART,6.0,0
1,1908,5.0,15.0,8.0,31.0,36.0,7.0,59.00,-141.00,CAPE YAKATAGA,6.0,0
2,1909,9.0,8.0,17.0,49.0,48.0,7.4,52.50,-169.00,ALEUTIAN ISLANDS,5.0,0
3,1911,9.0,22.0,5.0,1.0,24.0,6.9,60.50,-149.00,PRINCE WILLIAM SOUND,8.0,0
4,1912,1.0,31.0,20.0,11.0,48.0,7.3,61.00,-147.50,VALDES,6.0,0
5,1912,6.0,7.0,9.0,55.0,54.0,6.4,59.00,-153.00,KANATAK,5.0,0
6,1912,6.0,10.0,16.0,6.0,6.0,7.0,59.00,-153.00,NaN,3.0,0
7,1912,7.0,7.0,7.0,57.0,36.0,7.4,64.00,-147.00,KENNECOTT,5.0,0
8,1912,7.0,7.0,21.0,57.0,36.0,7.4,64.00,-147.00,FAIRBANKS,6.0,0
9,1912,11.0,7.0,7.0,40.0,24.0,7.5,57.50,-155.00,SEWARDOTT,NaN,0


To convert date/time values to a datetime data type, we need real values for all missing ones. We'll set those missing time values to zero.

In [6]:
dataframe['HOUR'] = dataframe['HOUR'].fillna(value=0)
dataframe['MINUTE'] = dataframe['MINUTE'].fillna(value=0)
dataframe['SECOND'] = dataframe['SECOND'].fillna(value=0)
dataframe

,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,MAGNITUDE,LATITUDE,LONGITUDE,CITY,MMI,accuracy
0,1904,8.0,27.0,21.0,56.0,6.0,8.3,65.48,-150.30,RAMPART,6.0,0
1,1908,5.0,15.0,8.0,31.0,36.0,7.0,59.00,-141.00,CAPE YAKATAGA,6.0,0
2,1909,9.0,8.0,17.0,49.0,48.0,7.4,52.50,-169.00,ALEUTIAN ISLANDS,5.0,0
3,1911,9.0,22.0,5.0,1.0,24.0,6.9,60.50,-149.00,PRINCE WILLIAM SOUND,8.0,0
4,1912,1.0,31.0,20.0,11.0,48.0,7.3,61.00,-147.50,VALDES,6.0,0
5,1912,6.0,7.0,9.0,55.0,54.0,6.4,59.00,-153.00,KANATAK,5.0,0
6,1912,6.0,10.0,16.0,6.0,6.0,7.0,59.00,-153.00,NaN,3.0,0
7,1912,7.0,7.0,7.0,57.0,36.0,7.4,64.00,-147.00,KENNECOTT,5.0,0
8,1912,7.0,7.0,21.0,57.0,36.0,7.4,64.00,-147.00,FAIRBANKS,6.0,0
9,1912,11.0,7.0,7.0,40.0,24.0,7.5,57.50,-155.00,SEWARDOTT,NaN,0


Now convert the date/time columns into a datetime field

In [7]:
tcols = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'SECOND']
dataframe['time'] = dataframe['time'] = pd.to_datetime(dataframe[tcols])
dataframe = dataframe.sort_values('time')
dataframe

,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,MAGNITUDE,LATITUDE,LONGITUDE,CITY,MMI,accuracy,time
10,1727,11.0,10.0,3.0,40.0,0.0,NaN,42.80,-70.80,NEWBURY,8.0,60,1727-11-10 03:40:00
11,1732,9.0,16.0,16.0,0.0,0.0,NaN,45.50,-73.60,MONTREAL,9.0,60,1732-09-16 16:00:00
12,1737,2.0,17.0,21.0,30.0,0.0,NaN,42.40,-71.00,BOSTON,4.0,60,1737-02-17 21:30:00
13,1737,12.0,19.0,4.0,0.0,0.0,NaN,40.80,-74.00,NEW YORK,7.0,60,1737-12-19 04:00:00
14,1741,6.0,24.0,15.0,35.0,0.0,NaN,42.20,-71.20,BOSTON,5.0,60,1741-06-24 15:35:00
15,1744,6.0,14.0,15.0,15.0,0.0,NaN,42.60,-70.90,BOSTON,6.0,60,1744-06-14 15:15:00
16,1755,11.0,18.0,9.0,11.0,0.0,NaN,42.60,-70.00,BOSTON,8.0,60,1755-11-18 09:11:00
17,1755,11.0,18.0,10.0,29.0,0.0,NaN,42.60,-70.00,SCITUATE,7.0,60,1755-11-18 10:29:00
18,1755,11.0,23.0,1.0,27.0,0.0,NaN,42.70,-70.30,BOSTON,5.0,60,1755-11-23 01:27:00
19,1758,4.0,25.0,2.0,30.0,0.0,NaN,38.90,-76.50,ANNAPOLIS,4.0,60,1758-04-25 02:30:00


## Performing Association

The association function looks like this:

    associate(
        dataframe,
        time_column='time',
        lat_column='latitude',
        lon_column='longitude',
        mag_column='magnitude',
        time_tol_secs=16,
        dist_tol_km=100,
        mag_tol=0.5,
    )

Notice there are three tolerance fields that we can set, for time, distance, and magnitude. Whenever we compare *origin* (time, lat, lon, depth, magnitude) quantities between events in two catalogues, we must take into account the differences between those quantities in the two different catalogues. Note that it may take some exploration and analysis of the two catalogues to determine what these values are.

In a data set like this where we *know* the resolution of the data is not as high as we might desire, we need to take care to add that uncertainty to our tolerance fields.


In the cell below we'll extract subsets of our input catalogue by accuracy value (we could do this in a loop with the groupby() method, but we'll leave that as an exercise for the reader.)

Rows where seconds are present:

In [8]:
seconds_frame = dataframe[dataframe['accuracy'] == 0]
associated_seconds, _ = associate(seconds_frame, 
                                                  time_tol_secs=60, 
                                                  mag_column='MAGNITUDE', 
                                                  lat_column='LATITUDE',
                                                  lon_column='LONGITUDE'
                                                 )
associated_seconds

,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,MAGNITUDE,LATITUDE,LONGITUDE,CITY,MMI,accuracy,time,comcat_id,comcat_time,comcat_latitude,comcat_longitude,comcat_depth,comcat_magnitude,comcat_score
1,1908,5.0,15.0,8.0,31.0,36.0,7.0,59.0,-141.0,CAPE YAKATAGA,6.0,0,1908-05-15 08:31:36,ushis474,1908-05-15 08:31:36,59.0,-141.0,NaN,7.0,0.0
3,1911,9.0,22.0,5.0,1.0,24.0,6.9,60.5,-149.0,PRINCE WILLIAM SOUND,8.0,0,1911-09-22 05:01:24,ushis524,1911-09-22 05:01:24,60.5,-149.0,60.0,6.9,0.0
4,1912,1.0,31.0,20.0,11.0,48.0,7.3,61.0,-147.5,VALDES,6.0,0,1912-01-31 20:11:48,ushis529,1912-01-31 20:11:48,61.0,-147.5,80.0,7.0,0.0
5,1912,6.0,7.0,9.0,55.0,54.0,6.4,59.0,-153.0,KANATAK,5.0,0,1912-06-07 09:55:54,ushis532,1912-06-07 09:55:54,59.0,-153.0,NaN,6.4,0.0
6,1912,6.0,10.0,16.0,6.0,6.0,7.0,59.0,-153.0,NaN,3.0,0,1912-06-10 16:06:06,ushis533,1912-06-10 16:06:06,59.0,-153.0,NaN,6.9,0.0
9,1912,11.0,7.0,7.0,40.0,24.0,7.5,57.5,-155.0,SEWARDOTT,NaN,0,1912-11-07 07:40:24,ushis541,1912-11-07 07:40:24,57.5,-155.0,90.0,7.3,0.0


Rows where seconds are absent. Note here that all of our magnitude values are also missing. The associate function works best if provided time, location, and magnitude data, but it can attempt to associate events even if location and magnitude data are not present.

In [9]:
minutes_frame = dataframe[dataframe['accuracy'] == 60]
associated_minutes, _ = associate(minutes_frame, 
                                                  time_tol_secs=300, 
                                                  mag_column='MAGNITUDE', 
                                                  lat_column='LATITUDE',
                                                  lon_column='LONGITUDE'
                                                 )
associated_minutes

,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,MAGNITUDE,LATITUDE,LONGITUDE,CITY,MMI,accuracy,time,comcat_id,comcat_time,comcat_latitude,comcat_longitude,comcat_depth,comcat_magnitude,comcat_score
10,1727,11.0,10.0,3.0,40.0,0.0,NaN,42.8,-70.8,NEWBURY,8.0,60,1727-11-10 03:40:00,official17271110034000000,1727-11-10 03:40:00,42.8417,-70.9788,NaN,5.5,0.0
11,1732,9.0,16.0,16.0,0.0,0.0,NaN,45.5,-73.6,MONTREAL,9.0,60,1732-09-16 16:00:00,ushis4,1732-09-16 16:00:00,45.5000,-73.6000,NaN,6.3,0.0
15,1744,6.0,14.0,15.0,15.0,0.0,NaN,42.6,-70.9,BOSTON,6.0,60,1744-06-14 15:15:00,ushis6,1744-06-14 15:15:00,42.6000,-70.9000,NaN,4.7,0.0
16,1755,11.0,18.0,9.0,11.0,0.0,NaN,42.6,-70.0,BOSTON,8.0,60,1755-11-18 09:11:00,ushis7,1755-11-18 09:11:35,42.7000,-70.3000,NaN,5.9,0.0


In the cases above, there are no alternate events that matched all of the input criteria. Below is an example where we have events that fall within a large time window. We will see that the score for the chosen event is lower than the scores for the other candidate events. 

In [20]:
ambiguous = pd.DataFrame({'time': [datetime(2019, 7, 6, 4, 20, 17)],
                              'latitude': [35.78],
                              'longitude': [-117.614],
                              'magnitude': 4.3})
ambiguous

,time,latitude,longitude,magnitude
0,2019-07-06 04:20:17,35.78,-117.614,4.3


In [21]:
associated, alternates = associate(ambiguous, time_tol_secs=120)
associated[['time','latitude','longitude','comcat_time','comcat_latitude','comcat_longitude','comcat_score']]

,time,latitude,longitude,comcat_time,comcat_latitude,comcat_longitude,comcat_score
0,2019-07-06 04:20:17,35.78,-117.614,2019-07-06 04:19:15.480,35.785,-117.6175,0.781111


In [22]:
alternates[['time','latitude','longitude','score']]

,time,latitude,longitude,score
1,2019-07-06 04:19:54.940,35.885000,-117.717000,1.157689
2,2019-07-06 04:19:56.010,35.580833,-117.404167,1.204837
3,2019-07-06 04:21:35.820,35.778667,-117.612167,1.037576
